In [1]:
import ARESlatticeStage3v1_9 as ares
import cheetah
import xobjects as xo
import xpart as xp
import numpy as np
import xtrack as xt
from ocelot.adaptors.astra2ocelot import astraBeam2particleArray

[INFO    ] csr.py: module PYFFTW is not installed. Install it to speed up calculation.


initializing ocelot...
import: module PYFFTW is not installed. Install it to speed up calculation


In [2]:
segment = cheetah.Segment.from_ocelot(ares.cell, warnings=False).subcell(
    "AREASOLA1", "AREABSCR1"
)
segment

Segment(elements=ModuleList(
  (0): Marker(name='AREASOLA1')
  (1): Drift(length=tensor(0.1750))
  (2): Quadrupole(length=tensor(0.1220), k1=tensor(0.), misalignment=tensor([0., 0.]), tilt=tensor(0.), name='AREAMQZM1')
  (3): Drift(length=tensor(0.4280))
  (4): Quadrupole(length=tensor(0.1220), k1=tensor(0.), misalignment=tensor([0., 0.]), tilt=tensor(0.), name='AREAMQZM2')
  (5): Drift(length=tensor(0.2040))
  (6): VerticalCorrector(length=tensor(0.0200), angle=tensor(0.), name='AREAMCVM1')
  (7): Drift(length=tensor(0.2040))
  (8): Quadrupole(length=tensor(0.1220), k1=tensor(0.), misalignment=tensor([0., 0.]), tilt=tensor(0.), name='AREAMQZM3')
  (9): Drift(length=tensor(0.1790))
  (10): HorizontalCorrector(length=tensor(0.0200), angle=tensor(0.), name='AREAMCHM1')
  (11): Drift(length=tensor(0.4500))
  (12): Screen(resolution=tensor([2448, 2040]), pixel_size=tensor([3.5488e-06, 2.5003e-06]), binning=tensor(1), misalignment=tensor([0., 0.]), is_active=False, name='AREABSCR1')
), name

In [3]:
line = xt.Line(
    elements=[
        xt.Marker(),
        xt.Drift(length=0.1750),
        xt.Quadrupole(length=0.1220, k1=10.0),
        xt.Drift(length=0.4280),
        xt.Quadrupole(length=0.1220, k1=-9.0),
        xt.Drift(length=0.2040),
        xt.SimpleThinBend(length=0.0200, hxl=1e-3),
        xt.Drift(length=0.2040),
        xt.Quadrupole(length=0.1220, k1=-8.0),
        xt.Drift(length=0.1790),
        xt.SimpleThinBend(length=0.0200, hxl=-2e-3),
        xt.Drift(length=0.4500),
        xt.BeamPositionMonitor(),
        xt.BeamSizeMonitor(),
    ],
    element_names=[element.name for element in segment.elements]
    + [f"{segment.elements[-1].name}_2"],
)

# Choose a context
context = xo.ContextCpu()  # For CPU
# context = xo.ContextCupy()      # For CUDA GPUs

# Transfer lattice on context and compile tracking code
line.build_tracker(_context=context)

Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.


In [4]:
particle_array = astraBeam2particleArray("ACHIP_EA1_2021.1351.001")

particles = xp.Particles(
    x=particle_array.x(),
    px=particle_array.px(),
    y=particle_array.y(),
    py=particle_array.py(),
    zeta=particle_array.tau(),
    delta=particle_array.pz,
    mass0=xp.ELECTRON_MASS_EV,
    energy0=particle_array.E * 1e9,
    q0=-1,
)
particles

Astra to Ocelot: charge =  5.000000000001694e-13  C
Astra to Ocelot: particles number =  100000
Astra to Ocelot: energy =  0.10731590244355117  GeV
Astra to Ocelot: s pos =  13.50890653269  m


Particles(_capacity=100000, _num_active_particles=100000, _num_lost_particles=0, start_tracking_at_element=-1, q0=-1.0, mass0=510998.9499961642, _p0c=[1.07314686e+08 1.07314686e+08 1.07314686e+08 ... 1.07314686e+08
 1.07314686e+08 1.07314686e+08], _gamma0=[210.0119823 210.0119823 210.0119823 ... 210.0119823 210.0119823
 210.0119823], _beta0=[0.99998866 0.99998866 0.99998866 ... 0.99998866 0.99998866 0.99998866], s=[0. 0. 0. ... 0. 0. 0.], zeta=[-0.00000000e+00  2.18696679e-06  6.85092638e-07 ...  8.31485105e-07
  1.18904531e-06  6.57932034e-06], _ptau=[0.99999433 0.99896866 1.00117994 ... 0.99944702 1.00143116 0.99817854], _delta=[1.         0.99897433 1.00118561 ... 0.99945269 1.00143684 0.99818421], _rpp=[0.5        0.50025655 0.49970377 ... 0.50013687 0.49964105 0.50045436], _rvv=[1.0000085  1.0000085  1.00000851 ... 1.0000085  1.00000851 1.0000085 ], chi=[1. 1. 1. ... 1. 1. 1.], charge_ratio=[1. 1. 1. ... 1. 1. 1.], weight=[1. 1. 1. ... 1. 1. 1.], pdg_id=[0 0 0 ... 0 0 0], particle

In [5]:
%%timeit
outgoing_particles = line.track(particles)

196 µs ± 1.1 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [6]:
%%timeit
outgoing_particles = line.track(particles.copy())

2.82 ms ± 3.33 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [7]:
%%timeit
outgoing_particles = line.track(particles.copy())
line["AREAMQZM1"].k1 = np.random.rand() * 30.0

2.83 ms ± 3.84 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [8]:
%%timeit
line.discard_tracker()
line.build_tracker(_context=context)
outgoing_particles = line.track(particles)

Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.
Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.
Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.
Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.
Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.
Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.
Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.
Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.
5.55 s ± 21.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
line = xt.Line(
    elements=[
        xt.Marker(),
        xt.Drift(length=0.1750),
        xt.Quadrupole(length=0.1220),
        xt.Drift(length=0.4280),
        xt.Quadrupole(length=0.1220),
        xt.Drift(length=0.2040),
        # xt.SimpleThinBend(length=0.0200, hxl=1e-3),
        xt.Bend(length=0.0200),
        xt.Drift(length=0.2040),
        xt.Quadrupole(length=0.1220),
        xt.Drift(length=0.1790),
        # xt.SimpleThinBend(length=0.0200, hxl=-2e-3),
        xt.Bend(length=0.0200),
        xt.Drift(length=0.4500),
        xt.BeamPositionMonitor(),
        xt.BeamSizeMonitor(),
    ],
    element_names=[element.name for element in segment.elements]
    + [f"{segment.elements[-1].name}_2"],
)

# Choose a context
context = xo.ContextCpu()  # For CPU
# context = xo.ContextCupy()      # For CUDA GPUs

# Transfer lattice on context and compile tracking code
line.build_tracker(_context=context)

Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.


In [10]:
%%timeit
outgoing_particles = line.track(particles.copy())

2.87 ms ± 7.11 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [11]:
line = xt.Line(
    elements=[
        xt.Marker(),
        xt.Drift(length=0.1750),
        xt.Quadrupole(length=0.1220, k1=10.0),
        xt.Drift(length=0.4280),
        xt.Quadrupole(length=0.1220, k1=-9.0),
        xt.Drift(length=0.2040),
        # xt.SimpleThinBend(length=0.0200, hxl=1e-3),
        xt.Bend(length=0.0200, k0=0.1),
        xt.Drift(length=0.2040),
        xt.Quadrupole(length=0.1220, k1=-8.0),
        xt.Drift(length=0.1790),
        # xt.SimpleThinBend(length=0.0200, hxl=-2e-3),
        xt.Bend(length=0.0200, k0=-0.1),
        xt.Drift(length=0.4500),
        xt.BeamPositionMonitor(),
        xt.BeamSizeMonitor(),
    ],
    element_names=[element.name for element in segment.elements]
    + [f"{segment.elements[-1].name}_2"],
)

# Choose a context
# context = xo.ContextCpu()  # For CPU
context = xo.ContextCupy()  # For CUDA GPUs

# Transfer lattice on context and compile tracking code
line.build_tracker(_context=context)

In [12]:
particles = xp.Particles(
    x=particle_array.x(),
    px=particle_array.px(),
    y=particle_array.y(),
    py=particle_array.py(),
    zeta=particle_array.tau(),
    delta=particle_array.pz,
    mass0=xp.ELECTRON_MASS_EV,
    energy0=particle_array.E * 1e9,
    q0=-1,
    _context=context,
)
particles

Particles(_capacity=100000, _num_active_particles=-1, _num_lost_particles=-1, start_tracking_at_element=-1, q0=-1.0, mass0=510998.9499961642, _p0c=[1.07314686e+08 1.07314686e+08 1.07314686e+08 ... 1.07314686e+08
 1.07314686e+08 1.07314686e+08], _gamma0=[210.0119823 210.0119823 210.0119823 ... 210.0119823 210.0119823
 210.0119823], _beta0=[0.99998866 0.99998866 0.99998866 ... 0.99998866 0.99998866 0.99998866], s=[0. 0. 0. ... 0. 0. 0.], zeta=[-0.00000000e+00  2.18696679e-06  6.85092638e-07 ...  8.31485105e-07
  1.18904531e-06  6.57932034e-06], _ptau=[0.99999433 0.99896866 1.00117994 ... 0.99944702 1.00143116 0.99817854], _delta=[1.         0.99897433 1.00118561 ... 0.99945269 1.00143684 0.99818421], _rpp=[0.5        0.50025655 0.49970377 ... 0.50013687 0.49964105 0.50045436], _rvv=[1.0000085  1.0000085  1.00000851 ... 1.0000085  1.00000851 1.0000085 ], chi=[1. 1. 1. ... 1. 1. 1.], charge_ratio=[1. 1. 1. ... 1. 1. 1.], weight=[1. 1. 1. ... 1. 1. 1.], pdg_id=[0 0 0 ... 0 0 0], particle_id

In [15]:
%%timeit
outgoing_particles = line.track(particles.copy())

571 µs ± 1.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
